In [51]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [52]:
path = "/home/harshit/Project/Returns/averages/"
factors = pd.read_csv(path+"FourFactors.csv")

In [53]:
#Making single dataframe having entropy factor values for all years
ent_ret1 = pd.read_csv(path+"average2013.csv").drop([0])
ent_ret1.Date = ent_ret1.Date.apply(lambda x: x.replace("-",""))
ent_ret2 = pd.read_csv(path+"average2014.csv").drop([0])
ent_ret2.Date = ent_ret2.Date.apply(lambda x: x.replace("-",""))
ent_ret3 = pd.read_csv(path+"average2015.csv").drop([0])
ent_ret3.Date = ent_ret3.Date.apply(lambda x: x.replace("-",""))
ent_ret4 = pd.read_csv(path+"average2016.csv").drop([0])
ent_ret4.Date = ent_ret4.Date.apply(lambda x: x.replace("-",""))
ent_ret5 = pd.read_csv(path+"average2017.csv").drop([0])
ent_ret5.Date = ent_ret5.Date.apply(lambda x: x.replace("-",""))

In [54]:
#Making single dataframe having values for all years FOR RENYI_ENTROPY
ren_ent_ret1 = pd.read_csv(path+"renyi_average2013.csv").drop([0])
ren_ent_ret1.Date = ent_ret1.Date.apply(lambda x: x.replace("-",""))
ren_ent_ret2 = pd.read_csv(path+"renyi_average2014.csv").drop([0])
ren_ent_ret2.Date = ent_ret2.Date.apply(lambda x: x.replace("-",""))
ren_ent_ret3 = pd.read_csv(path+"renyi_average2015.csv").drop([0])
ren_ent_ret3.Date = ent_ret3.Date.apply(lambda x: x.replace("-",""))
ren_ent_ret4 = pd.read_csv(path+"renyi_average2016.csv").drop([0])
ren_ent_ret4.Date = ent_ret4.Date.apply(lambda x: x.replace("-",""))
ren_ent_ret5 = pd.read_csv(path+"renyi_average2017.csv").drop([0])
ren_ent_ret5.Date = ent_ret5.Date.apply(lambda x: x.replace("-",""))

In [71]:
entire = pd.concat([ent_ret1,ent_ret2,ent_ret3,ent_ret4,ent_ret5],ignore_index=True)
entire["Date"] = entire["Date"].apply(pd.to_numeric) # converting datatype of dates to numeric

#FOR RENYI_ENTROPY
ren_entire = pd.concat([ren_ent_ret1,ren_ent_ret2,ren_ent_ret3,ren_ent_ret4,ren_ent_ret5],ignore_index=True)
ren_entire["Date"] = entire["Date"].apply(pd.to_numeric) # converting datatype of dates to numeric

In [72]:
#joining frames to have a single dataframe having all the factors together
all_factors= pd.merge(factors,entire,on="Date") # Joining dataframe of factors and entropyfactor
#all_factors.to_csv("/home/harshit/Project/allfactors.csv",index=False)

# for reney entropy
ren_all_factors= pd.merge(factors,ren_entire,on="Date")
#ren_all_factors.to_csv("/home/harshit/Project/allfactors.csv",index=False)

In [ ]:
#Making company vise csv's of returns for all years
eq_path = "/home/harshit/Project/all_price_by_company/"
eq_list = os.listdir("/home/harshit/Project/all_price_by_company/")
eq_list = pd.DataFrame(eq_list,columns=["Equities"])
return_path = "/home/harshit/Project/risk_free_ret/all_ret_by_company/"
for i in eq_list["Equities"]:
    df2 = pd.DataFrame(columns=["Date","Return"])
    name = i.split('.')[0]
    data = pd.read_csv(eq_path+i)
    for j in range(len(data)):
        date = data.iloc[j]["Date"]
        previous = data.iloc[j-1][name]
        new = data.iloc[j][name]
        ret = ((previous-new)/previous)*100
        df = pd.DataFrame([[date,ret]],columns=["Date","Return"])
        df2 = df2.append(df,ignore_index=True)      # A DF having return data for all dates
        df2.at[0, 'Return'] = 0
        df2.to_csv(return_path+i,index=False)

In [74]:
#Applying Linear Regression to equities
eq_path = "/home/harshit/Project/risk_free_ret/all_ret_by_company/"
model_car_path = "/home/harshit/Project/car_models/"  # PATH FOR CARHART MODELS

ent_path = "/home/harshit/Project/ent_models/" # PATH FOR ENTROPY MODELS
model3_path = "/home/harshit/Project/complete_models/" # PATH FOR MODELS WITH ALL FACTORS USED
eq_list = os.listdir(eq_path) #a list of all equities
eq_list = pd.DataFrame(eq_list,columns=["Equities"])

r2score = pd.DataFrame(columns=["Symbol","ent_insample","car_insample","ent_outsample","car_outsample"])
pvals   = pd.DataFrame(columns=["Symbol","SMB %","HML %","WML %","Rm-Rf %","high-low","F-statistics","P(F-value)","Adj-r_sq"])
for i in eq_list["Equities"]:   #making model for each company over the entire period
    equity = pd.read_csv(eq_path+i)
    equity=equity.drop([0])
    #q = equity["Return"].quantile(0.99)
    #equity= equity[np.abs(equity["Return"]) < q]
    #equity=equity[np.abs(equity.Return-equity.Return.mean()) <= (2*equity.Return.std())] #removing outliers
    all_data = pd.merge(all_factors,equity,on="Date") # Making a dataframe by date,having all factors and equity returns
    ren_all_data = pd.merge(ren_all_factors,equity,on="Date") # FOR RENYI_ENTROPY
    symbol = i.split('.')[0]  #Removing ".csv" to get file name
    all_data.rename(columns={'Return':symbol}, inplace=True)
   
    X1= all_data[["SMB %","HML %","WML %","Rm-Rf %","high-low"]] #Making X variable for entropy included model
    X1 = sm.add_constant(X1)
    X2= all_data[["SMB %","HML %","WML %","Rm-Rf %"]] #Making X variable for Carhart 4 factor model
    X2 = sm.add_constant(X2)
    y = all_data[[symbol]]
    X_ent_train, X_ent_test, y_ent_train, y_ent_test = train_test_split(X1,y, test_size=0.2)
    X_car_train, X_car_test, y_car_train, y_car_test = train_test_split(X2,y, test_size=0.2)
    #Making entropy model
    model1= sm.OLS(y_ent_train,X_ent_train).fit()  # model with entropy factor           
    model1_name = ent_path+symbol+".sav"
    
    #Making model without entropy
    model2= sm.OLS(y_car_train,X_car_train,hasconst=True).fit()   #model for carhart 4 factor
    model2_name = model_car_path+symbol+".sav"
    
    #making a model without test-train divide
    model3 = sm.OLS(y,X1,hasconst=True).fit()
    model3_name = model3_path+symbol+".sav"
    
    pickle.dump(model1,open(model1_name,"wb"))
    pickle.dump(model2,open(model2_name,"wb"))
    pickle.dump(model3,open(model3_name,"wb"))
    #comparing model 1 and 2 by saving r2 scores
    ent_insample = r2_score(y_ent_train,model1.predict(X_ent_train))*100
    car_insample = r2_score(y_car_train,model2.predict(X_car_train))*100
    ent_outsample= r2_score(y_ent_test,model1.predict(X_ent_test))*100
    car_outsample = r2_score(y_car_test,model2.predict(X_car_test))*100
    df = pd.DataFrame([[symbol,ent_insample,car_insample,ent_outsample,car_outsample]],
                      columns=["Symbol","ent_insample","car_insample","ent_outsample","car_outsample"])
    r2score= r2score.append(df,ignore_index=True)
    #saving model statistics and pvalues of variables
    a=pd.DataFrame(model3.pvalues).T
    a.insert(0,"Symbol",symbol)
    a.insert(6,"F-statistics",model3.fvalue)
    a.insert(7,"P(F-value)",model3.f_pvalue)
    a.insert(8,"Adj-r_sq",model3.rsquared_adj)
    pvals=pvals.append(a,ignore_index=True)
    pvals=pvals.round(4)                     #round() removes scientific notation

r2score.to_csv("/home/harshit/Project/r2scores.csv")
pvals.to_csv("/home/harshit/Project/pvalues.csv")

In [70]:
all

,SMB %,HML %,WML %,Rm-Rf %,high-low


In [59]:
#Applying Linear Regression using multiple set of independents without sampeling
eq_path = "/home/harshit/Project/risk_free_ret/all_ret_by_company/"
model_car_path = "/home/harshit/Project/car_models/" 
model_ent_path = "/home/harshit/Project/ent_models/"
eq_list = os.listdir("/home/harshit/Project/risk_free_ret/all_ret_by_company/") #a list of all equities
eq_list = pd.DataFrame(eq_list,columns=["Equities"])

pvals_rm_rf   = pd.DataFrame(columns=["Symbol","Rm-Rf %","Adj-r_sq"])
pvals_rm_smb   = pd.DataFrame(columns=["Symbol","Rm-Rf %","SMB %","Adj-r_sq"])
pvals_rm_smb_hml   = pd.DataFrame(columns=["Symbol","Rm-Rf %","SMB %","HML %","Adj-r_sq"])
for i in eq_list["Equities"]:   #making model for each company over the entire period
    equity = pd.read_csv(eq_path+i)
    equity=equity.drop([0])
    all_data = pd.merge(all_factors,equity,on="Date") # Making a dataframe by date,having all factors and equity returns
    symbol = i.split('.')[0]  #Removing ".csv" to get file name
    all_data.rename(columns={'Return':symbol}, inplace=True)
    X1= all_data[["Rm-Rf %"]] #Making X variable for entropy included model
    X1 = sm.add_constant(X1)
    X2= all_data[["Rm-Rf %","high-low"]]
    X2= sm.add_constant(X2)
    X3=all_data[["Rm-Rf %","SMB %","HML %"]]
    X3= sm.add_constant(X3)
    
    y = all_data[[symbol]]
    model3 = sm.OLS(y,X1,hasconst=True).fit()
    model4 = sm.OLS(y,X2,hasconst=True).fit()
    model5 = sm.OLS(y,X3,hasconst=True).fit()
    
    a=pd.DataFrame(model3.pvalues).T
    a2=pd.DataFrame(model4.pvalues).T
    a3=pd.DataFrame(model5.pvalues).T
    a.insert(0,"Symbol",symbol)
    a2.insert(0,"Symbol",symbol)
    a3.insert(0,"Symbol",symbol)
    a.insert(2,"Adj-r_sq",model3.rsquared_adj)
    a2.insert(2,"Adj-r_sq",model4.rsquared_adj)
    a3.insert(2,"Adj-r_sq",model5.rsquared_adj)
    pvals_rm_rf=pvals_rm_rf.append(a,ignore_index=True)
    pvals_rm_rf=pvals_rm_rf.round(4)                    #round() removes scientific notation
    pvals_rm_ent=pvals_rm_ent.append(a2,ignore_index=True)
    pvals_rm_ent=pvals_rm_ent.round(4)
    pvals_rm_smb_hml=pvals_rm_smb_hml.append(a3,ignore_index=True)
    pvals_rm_smb_hml=pvals_rm_smb_hml.round(4)

pvals_rm_rf.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_rf.csv",index=False)
pvals_rm_ent.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_ent.csv",index=False)
pvals_rm_smb_hml.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_smb_hml.csv",index=False)

In [47]:
 model1_name = model_ent_path+symbol+".sav"

,Date,SMB %,HML %,WML %,Rm %,Rf %,Rm-Rf %,high-low
1,20130102,-0.035327,0.096808,0.210578,0.664796,0.031492,0.633303,0.175220
2,20130103,-0.620298,1.464153,-0.281452,0.471858,0.031159,0.440698,-0.097420
3,20130104,-0.306602,0.832344,-0.020690,0.325108,0.031159,0.293949,0.001715
4,20130107,0.707425,0.213944,-0.876943,-0.059122,0.031159,-0.090282,0.421978
5,20130108,-0.432643,0.539166,0.503192,0.072671,0.031159,0.041511,-0.918361
6,20130109,0.388757,0.018923,0.255774,-0.323504,0.031159,-0.354663,0.326931
7,20130110,-0.154430,-0.804478,0.315932,-0.189251,0.031011,-0.220262,-0.643967
8,20130111,0.366848,-1.540808,-3.757202,-0.859876,0.031011,-0.890887,-0.384956
9,20130114,-1.359168,1.340599,-0.913450,1.183552,0.031011,1.152541,0.273918
10,20130115,-0.761608,0.785382,0.732481,0.333292,0.031011,0.302281,0.541376


In [78]:
#Yearly models
path = "/home/harshit/Project/Returns/averages/"
model4_path = "/home/harshit/Project/complete_models/2017/"
factors = pd.read_csv(path+"FourFactors.csv")
ent_ret = pd.read_csv(path+"average2017.csv").drop([0])
ent_ret= ent_ret.drop(["high","low"],axis=1)
ent_ret.Date = ent_ret.Date.apply(lambda x: x.replace("-",""))
ent_ret["Date"] = ent_ret["Date"].apply(pd.to_numeric)
all_factors= pd.merge(factors,ent_ret,on="Date",left_index=True)
all_factors
#Making yearly models for companies and saving them
##############################################################################################################
eq_path = "/home/harshit/Project/risk_free_ret/2017/"
eq_list = os.listdir("/home/harshit/Project/risk_free_ret/2017/") #a list of all equities
eq_list = pd.DataFrame(eq_list,columns=["Equities"])
pvals_rm_rf   = pd.DataFrame(columns=["Symbol","Rm-Rf %","Adj-r_sq"])
pvals_rm_ent   = pd.DataFrame(columns=["Symbol","Rm-Rf %","SMB %","Adj-r_sq"])
pvals_rm_smb_hml   = pd.DataFrame(columns=["Symbol","Rm-Rf %","SMB %","HML %","Adj-r_sq"])
pvals_entire   = pd.DataFrame(columns=["Symbol","Rm-Rf %","SMB %","HML %","WML %","ent","Adj-r_sq"])

for i in eq_list["Equities"]:   #making model for each company over the entire period
    equity = pd.read_csv(eq_path+i)
    all_data = pd.merge(all_factors,equity,on="Date") # Making a dataframe by date,having all factors and equity returns
    symbol = i.split('.')[0]  #Removing ".csv" to get file name
    all_data.rename(columns={'Return':symbol}, inplace=True)
    X1= all_data[["Rm-Rf %"]] #Making X variable for entropy included model
    X1= sm.add_constant(X1)
    X2= all_data[["Rm-Rf %","high-low"]]
    X2= sm.add_constant(X2)
    X3= all_data[["Rm-Rf %","SMB %","HML %"]]
    X3= sm.add_constant(X3)
    X4= all_data[["Rm-Rf %","SMB %","HML %","WML %","high-low"]]
    X4= sm.add_constant(X4)
    y = all_data[[symbol]]
    
    model1 = sm.OLS(y,X1,hasconst=True).fit()
    model2 = sm.OLS(y,X2,hasconst=True).fit()
    model3 = sm.OLS(y,X3,hasconst=True).fit()
    model4 = sm.OLS(y,X4,hasconst=True).fit()
    model4_name = model4_path+symbol+".sav"
    pickle.dump(model4,open(model4_name,"wb"))
    
    a=pd.DataFrame(model1.pvalues).T
    a2=pd.DataFrame(model2.pvalues).T
    a3=pd.DataFrame(model3.pvalues).T
    a4=pd.DataFrame(model3.pvalues).T
    a.insert(0,"Symbol",symbol)
    a2.insert(0,"Symbol",symbol)
    a3.insert(0,"Symbol",symbol)
    a4.insert(0,"Symbol",symbol)
    
    a.insert(2,"Adj-r_sq",model1.rsquared_adj)
    a2.insert(2,"Adj-r_sq",model2.rsquared_adj)
    a3.insert(2,"Adj-r_sq",model3.rsquared_adj)
    a4.insert(2,"Adj-r_sq",model4.rsquared_adj)
    
    pvals_rm_rf=pvals_rm_rf.append(a,ignore_index=True)
    pvals_rm_rf=pvals_rm_rf.round(4)                    #round() removes scientific notation
    pvals_rm_ent=pvals_rm_ent.append(a2,ignore_index=True)
    pvals_rm_ent=pvals_rm_ent.round(4)
    pvals_rm_smb_hml=pvals_rm_smb_hml.append(a3,ignore_index=True)
    pvals_rm_smb_hml=pvals_rm_smb_hml.round(4)
    pvals_entire=pvals_entire.append(a4,ignore_index=True)
    pvals_entire=pvals_entire.round(4)
    
pvals_rm_rf.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_rf_2017.csv",index=False)
pvals_rm_ent.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_ent_2017.csv",index=False)
pvals_rm_smb_hml.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_rm_smb_hml_2017.csv",index=False)
pvals_entire.to_csv("/home/harshit/Project/pvals_and_r2/pvalues_entire_2017.csv",index=False)

In [60]:
all_data

,Date,SMB %,HML %,WML %,Rm %,Rf %,Rm-Rf %,high,low,high-low,HTMEDIA
0,20130103,-0.620298,1.464153,-0.281452,0.471858,0.031159,0.440698,-0.327885,-0.230464,-0.097420,1.227920
1,20130104,-0.306602,0.832344,-0.020690,0.325108,0.031159,0.293949,0.047579,0.045865,0.001715,0.606408
2,20130107,0.707425,0.213944,-0.876943,-0.059122,0.031159,-0.090282,0.231463,-0.190515,0.421978,-2.844585
3,20130108,-0.432643,0.539166,0.503192,0.072671,0.031159,0.041511,-0.365829,0.552532,-0.918361,0.016848
4,20130109,0.388757,0.018923,0.255774,-0.323504,0.031159,-0.354663,0.119220,-0.207711,0.326931,0.833394
5,20130110,-0.154430,-0.804478,0.315932,-0.189251,0.031011,-0.220262,0.023738,0.667705,-0.643967,-4.633725
6,20130111,0.366848,-1.540808,-3.757202,-0.859876,0.031011,-0.890887,0.529911,0.914867,-0.384956,2.331193
7,20130114,-1.359168,1.340599,-0.913450,1.183552,0.031011,1.152541,-0.712348,-0.986266,0.273918,-0.173326
8,20130115,-0.761608,0.785382,0.732481,0.333292,0.031011,0.302281,0.109568,-0.431808,0.541376,1.911196
9,20130116,0.550375,-1.617517,-0.072497,-0.953585,0.031011,-0.984596,0.524730,1.260677,-0.735946,-3.364345


In [15]:
pvals_rm_ent

,Symbol,Rm-Rf %,SMB %,Adj-r_sq


In [144]:
a

,Symbol,Rm-Rf %,Adj-r_sq
0,3MINDIA,5.449117e-38,0.125512


In [114]:
previous_entropy = pd.read_csv("/home/harshit/Project/2017/entropy.csv")

In [27]:
X1= all_data[["SMB %","HML %","WML %","Rm-Rf %","high-low"]] #Making X variable for entropy included model
X1 = sm.add_constant(X1)
X2= all_data[["SMB %","HML %","WML %","Rm-Rf %"]] #Making X variable for Carhart 4 factor model
X2 = sm.add_constant(X2)

In [21]:
X1

,const,Rm-Rf %
0,1.0,0.440698
1,1.0,0.293949
2,1.0,-0.090282
3,1.0,0.041511
4,1.0,-0.354663
5,1.0,-0.220262
6,1.0,-0.890887
7,1.0,1.152541
8,1.0,0.302281
9,1.0,-0.984596


In [39]:
all_factors

,Date,SMB %,HML %,WML %,Rm %,Rf %,Rm-Rf %,high,low,high-low
0,20130102,-0.035327,0.096808,0.210578,0.664796,0.031492,0.633303,-0.128437,-0.303657,0.175220
1,20130103,-0.620298,1.464153,-0.281452,0.471858,0.031159,0.440698,-0.327885,-0.230464,-0.097420
2,20130104,-0.306602,0.832344,-0.020690,0.325108,0.031159,0.293949,0.047579,0.045865,0.001715
3,20130107,0.707425,0.213944,-0.876943,-0.059122,0.031159,-0.090282,0.231463,-0.190515,0.421978
4,20130108,-0.432643,0.539166,0.503192,0.072671,0.031159,0.041511,-0.365829,0.552532,-0.918361
5,20130109,0.388757,0.018923,0.255774,-0.323504,0.031159,-0.354663,0.119220,-0.207711,0.326931
6,20130110,-0.154430,-0.804478,0.315932,-0.189251,0.031011,-0.220262,0.023738,0.667705,-0.643967
7,20130111,0.366848,-1.540808,-3.757202,-0.859876,0.031011,-0.890887,0.529911,0.914867,-0.384956
8,20130114,-1.359168,1.340599,-0.913450,1.183552,0.031011,1.152541,-0.712348,-0.986266,0.273918
9,20130115,-0.761608,0.785382,0.732481,0.333292,0.031011,0.302281,0.109568,-0.431808,0.541376
